In [105]:
# 1) First: install Java, Spark and and run a local Spark session by just running this on Google Colab:

!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null   # !apt-get --> install java
!wget -q https://downloads.apache.org/spark/spark-3.1.3/spark-3.1.3-bin-hadoop3.2.tgz  # !wget  --> download file from url
!tar xf spark-3.1.3-bin-hadoop3.2.tgz  # !tar --> like unzip 
!pip install -q findspark  # !pip  --> instal a package, we cant import a library without installing it first, most libraries that we used were already installed

# This are INSTALLATION COMMANDS IN LINUX that we run in our collab space, it's similar to downloading programs an installing them on our computers
# installs Apache Spark 3.1.2, Java 8, and Findspark, a library that makes it easy for Python to find Spark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 3s (82.5 kB/s)
Reading package li

In [106]:
# 2) Second: set the locations where Spark and Java are installed to let know Colab where to find it.

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.3-bin-hadoop3.2"

In [107]:
# 3) Third: import spark libraries and use them

import findspark
findspark.init("spark-3.1.3-bin-hadoop3.2") # SPARK_HOME

from pyspark.sql import SparkSession

# Create the session - We need to remember to close it at the end
# The session is basically our connection to Spark layer in the Hadoop ecosystem
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import Row
from pyspark.sql import functions as F

In [108]:
# Code for connecting our google drive to this collab notebook
from google.colab import drive
drive.mount('/content/drive')

# After you finish, look at the menu (gui to the left) and check under "files" that your "drive" folder was added.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [175]:
!ls "/content/drive/MyDrive/spark"  # Check what files we have in the path
# Notice! we have "u.item" file in that folder

 au.csv			   de_state_cities.csv	      users2.csv
 books_df.csv		   in.csv		      users3.csv
 BX-Book-Ratings.csv	   morecolforbooks.csv	      users_df.csv
 BXBooks.csv		   ratings_df.csv	      users_file1.csv
 bxbookswithnewcols.csv    Spark.ipynb		      users_file2.csv
 BX-Users.csv		  'Spark - Questions.ipynb'   users_file3.csv
 canadacities.csv	   u.data		      users_file.csv
 city_country_dict.csv	   u.item		     'users_to finish_work.csv'
 country_city_dict.csv	   uk_cities.csv	      us-states-territories.csv
 Cust_Rev_Very_Short.csv   uscities.csv		      walmart_stock.csv
 data			   users1.csv


In [176]:
#reading csv file to dataframe
spark = SparkSession.builder.appName("ratings").getOrCreate()
cols =["user_id","isbn","book_ratings"]
df = spark.read.options(delimiter='"')\
  .csv("/content/drive/MyDrive/spark/BX-Book-Ratings.csv", header=True)
df.printSchema()
df.show()

root
 |-- _c0: string (nullable = true)
 |-- User-ID;: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- ;: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- Book-Rating: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- ,,: string (nullable = true)

+----+--------+----+----------+----+---+----+-----------+----+----+---+
| _c0|User-ID;| _c2|      ISBN| _c4|  ;| _c6|Book-Rating| _c8| _c9| ,,|
+----+--------+----+----------+----+---+----+-----------+----+----+---+
|null| 276725;|null|034545104X|null|  ;|null|          0|null|null| ,,|
|null| 276726;|null|0155061224|null|  ;|null|          5|null|null| ,,|
|null| 276727;|null|0446520802|null|  ;|null|          0|null|null| ,,|
|null| 276729;|null|052165615X|null|  ;|null|          3|null|null| ,,|
|null| 276729;|null|0521795028|null|  ;|null|          6|null|null| ,,|
|null| 276733;|null|20

In [177]:
df = df.drop('_c0',
 '_c2',
 '_c4',
 ';',
 '_c6',
 '_c8',
 '_c9',
 ',,')

In [178]:
df.columns

['User-ID;', 'ISBN', 'Book-Rating']

In [179]:
df = df.withColumnRenamed("User-ID;","User_id") \
    .withColumnRenamed("ISBN","isbn")\
    .withColumnRenamed("Book-Rating","book_ratings")
df.printSchema()


root
 |-- User_id: string (nullable = true)
 |-- isbn: string (nullable = true)
 |-- book_ratings: string (nullable = true)



In [180]:
df.show()

+-------+----------+------------+
|User_id|      isbn|book_ratings|
+-------+----------+------------+
|276725;|034545104X|           0|
|276726;|0155061224|           5|
|276727;|0446520802|           0|
|276729;|052165615X|           3|
|276729;|0521795028|           6|
|276733;|2080674722|           0|
|276736;|3257224281|           8|
|276737;|0600570967|           6|
|276744;|038550120X|           7|
|276745;| 342310538|          10|
|276746;|0425115801|           0|
|276746;|0449006522|           0|
|276746;|0553561618|           0|
|276746;|055356451X|           0|
|276746;|0786013990|           0|
|276746;|0786014512|           0|
|276747;|0060517794|           9|
|276747;|0451192001|           0|
|276747;|0609801279|           0|
|276747;|0671537458|           9|
+-------+----------+------------+
only showing top 20 rows



In [181]:
df = df.select("isbn","book_ratings", F.translate(F.col("user_id"), ";", "").alias("user_id"))

In [182]:
df.count()

1048575

In [183]:
df.groupBy("user_id").count().orderBy("count",ascending=False).show()

+-------+-----+
|user_id|count|
+-------+-----+
|  11676|13602|
| 198711| 7550|
| 153662| 6109|
|  98391| 5891|
|  35859| 5850|
| 212898| 4785|
| 278418| 4533|
|  76352| 3367|
| 110973| 3100|
| 235105| 3067|
| 230522| 2991|
|  16795| 2948|
| 234623| 2674|
|  36836| 2529|
|  52584| 2512|
| 245963| 2507|
| 204864| 2504|
|  55492| 2459|
| 185233| 2448|
| 171118| 2421|
+-------+-----+
only showing top 20 rows



In [184]:
# nulling the isbn that contain characters that are not numbers or x
df = df.withColumn("isbn", F.regexp_replace(F.col("isbn"), "[^0-9 ^xX]", ''))
df.filter(F.col("isbn") == '').count()

63

In [185]:
df =df.filter((F.length("isbn")==13)|(F.length("isbn")==10))
df.count()

1040134

In [186]:
#counting distinct users in ratings
df.select("user_id").distinct().count()

94387

In [187]:
#reading csv file to dataframe
spark = SparkSession.builder.appName("users").getOrCreate()
df_users = spark.read.options(delimiter=',')\
  .csv("/content/drive/MyDrive/spark/users_file3.csv/part*.csv", header=True)
df_users.printSchema()
df_users.show()

root
 |-- User_ID: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- AgeGroups: string (nullable = true)

+-------+---+--------------+---------------+--------------+-----------+
|User_ID|Age|          city|          state|       country|  AgeGroups|
+-------+---+--------------+---------------+--------------+-----------+
|      1| -1|           Nyc|       New York| United States|    Unknown|
|      2| 18|      Stockton|     California| United States|       Teen|
|      3| -1|        Moscow|Yukon Territory|        Russia|    Unknown|
|      4| 17|         Porto|        Unknown|      Portugal|       Teen|
|      5| -1|   Farnborough|          Hants|United Kingdom|    Unknown|
|      6| 61|  Santa Monica|     California| United States|     Adults|
|      7| -1|    Washington|             Dc| United States|    Unknown|
|      8| -1|       Timmins|        Ontario|   

In [188]:
#counting distinct users in users
df_users.select("User_ID").distinct().count()

278859

In [189]:
# inner join ratings and users on user_id 
df = df.join(df_users, df.user_id == df_users.User_ID, 'inner').drop(df_users.User_ID)

In [190]:
df.count()

1040134

In [191]:
df.show()

+----------+------------+-------+---+----------+----------+-------------+---------+
|      isbn|book_ratings|user_id|Age|      city|     state|      country|AgeGroups|
+----------+------------+-------+---+----------+----------+-------------+---------+
|0195153448|           0|      2| 18|  Stockton|California|United States|     Teen|
|1881320189|           7|      8| -1|   Timmins|   Ontario|       Canada|  Unknown|
|1575663937|           6|      8| -1|   Timmins|   Ontario|       Canada|  Unknown|
|1567407781|           6|      8| -1|   Timmins|   Ontario|       Canada|  Unknown|
|1558746218|           0|      8| -1|   Timmins|   Ontario|       Canada|  Unknown|
|1552041778|           5|      8| -1|   Timmins|   Ontario|       Canada|  Unknown|
|0887841740|           5|      8| -1|   Timmins|   Ontario|       Canada|  Unknown|
|080652121X|           0|      8| -1|   Timmins|   Ontario|       Canada|  Unknown|
|0771074670|           0|      8| -1|   Timmins|   Ontario|       Canada|  U

In [192]:
#counting distinct users
df.select("user_id").distinct().count()

94387

In [193]:
#reading csv file to dataframe
spark = SparkSession.builder.appName("books").getOrCreate()
df_books = spark.read.options(delimiter=',')\
  .csv("/content/drive/MyDrive/spark/bxbookswithnewcols.csv", header=True)
df_books.printSchema()
df_books.show()

root
 |-- isbn: string (nullable = true)
 |-- book_title: string (nullable = true)
 |-- book_author: string (nullable = true)
 |-- year_of_publication: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- category: string (nullable = true)
 |-- pagecount: string (nullable = true)
 |-- language: string (nullable = true)

+----------+--------------------+--------------------+-------------------+--------------------+--------------+---------+--------+
|      isbn|          book_title|         book_author|year_of_publication|           publisher|      category|pagecount|language|
+----------+--------------------+--------------------+-------------------+--------------------+--------------+---------+--------+
|0195153448| Classical Mythology|  Mark P. O. Morford|               2002|Oxford University...|Social Science|      808|      en|
|0002005018|        Clara Callan|Richard Bruce Wright|               2001|HarperFlamingo Ca...|     Actresses|      414|      en|
|006097312

In [194]:
#counting distinct isbn in books
df_books.select("isbn").distinct().count()

271360

In [195]:
#counting distinct ratings
df.select("isbn").distinct().count()

316246

In [196]:
# inner join rating(with users) and books on user_id
df = df.join(df_books, df.isbn == df_books.isbn, 'inner').drop(df_books.isbn)
df.count()

940834

In [197]:
#counting distinct isbn
df.select("isbn").distinct().count()

257702

In [198]:
df.show()

+----------+------------+-------+---+----------------+----------------+--------------+-----------+--------------------+--------------------+-------------------+--------------------+--------------------+---------+--------+
|      isbn|book_ratings|user_id|Age|            city|           state|       country|  AgeGroups|          book_title|         book_author|year_of_publication|           publisher|            category|pagecount|language|
+----------+------------+-------+---+----------------+----------------+--------------+-----------+--------------------+--------------------+-------------------+--------------------+--------------------+---------+--------+
|0002234947|           0|  39608| -1|          Ottawa|         Ontario|        Canada|    Unknown|Miss Hobbema Pageant|       W.P. Kinsella|               1990|Harpercollins Juv...|       Short stories|      200|      en|
|0002234947|           0| 219445| 17|       Saskatoon|    Saskatchewan|        Canada|       Teen|Miss Hobbema P

In [133]:
#creating 3 df: ratings, users, books that have perfect matches in users and isbns
#ratings_df = df.drop("Age","city","country","state","book_title",'book_author','year_of_publication','publisher','language','category','pagecount')
#users_df = df.drop('book_ratings','isbn',"city","state","book_title",'book_author','year_of_publication','publisher','language','category','pagecount')
#books_df = df.drop('user_id','book_ratings','city','state','country','Age')

In [199]:
ratings_df = df.select("isbn","user_id", "book_ratings").distinct()
users_df = df. select("user_id","Age","AgeGroups","city","state","country").distinct()
books_df= df. select ("isbn","book_title","book_author","year_of_publication","publisher","category","pagecount","language").distinct()

In [200]:
ratings_df.count()

940830

In [201]:
users_df.count()

83663

In [202]:
books_df.count()

257702

In [203]:
ratings_df.show()

+----------+-------+------------+
|      isbn|user_id|book_ratings|
+----------+-------+------------+
|0843920262| 276762|           0|
|3596122279| 276936|           5|
|0713911131| 277378|           0|
|067091021X| 277427|           0|
|0836221311| 277427|           8|
|0553279556| 277466|           0|
|0449907481| 277554|           0|
|0879052473| 277562|           8|
|0330484338| 277901|           8|
|0451206460| 277919|           9|
|1557850003| 277929|           0|
|0515135968| 278026|           8|
|0375727345| 278122|          10|
|0385319959| 278314|           7|
|0070145121| 278418|           0|
|0152006044| 278418|           0|
|0307030849| 278418|           0|
|0307101452| 278418|           0|
|0373117868| 278418|           5|
|0373122462| 278418|           0|
+----------+-------+------------+
only showing top 20 rows



In [204]:
ratings_df.groupBy("book_ratings").count().orderBy("count").show()

+------------+------+
|book_ratings| count|
+------------+------+
|           1|  1339|
|           2|  2138|
|           3|  4683|
|           4|  6862|
|           6| 28970|
|           5| 41816|
|           9| 55949|
|           7| 60807|
|          10| 65081|
|           8| 84123|
|           0|589062|
+------------+------+



In [205]:
ratings_df.groupBy("isbn").count().orderBy("count", ascending=False).show()

+----------+-----+
|      isbn|count|
+----------+-----+
|0971880107| 2262|
|0316666343| 1164|
|0385504209|  812|
|0312195516|  667|
|0060928336|  662|
|044023722X|  595|
|0142001740|  561|
|067976402X|  552|
|0671027360|  536|
|0446672211|  527|
|0316601950|  518|
|059035342X|  515|
|0375727345|  505|
|044021145X|  487|
|0452282152|  486|
|0440214041|  479|
|0804106304|  476|
|0440211727|  469|
|0060930535|  455|
|0345337662|  449|
+----------+-----+
only showing top 20 rows



In [206]:
users_df.show()

+-------+---+-----------+-----------+-------------------+--------------+
|user_id|Age|  AgeGroups|       city|              state|       country|
+-------+---+-----------+-----------+-------------------+--------------+
|   2355| -1|    Unknown|   Gatineau|             Quebec|        Canada|
|   2936| 30|     Adults|     London|             London|United Kingdom|
|   2964| -1|    Unknown|   Tualatin|             Oregon| United States|
|   3282| 52|     Adults|   Didsbury|            Alberta|        Canada|
|   3641| 34|     Adults|King George|           Virginia| United States|
|   3645| -1|    Unknown|   Siegburg|Nordrhein-westfalen|       Germany|
|   4193| -1|    Unknown|    Douglas|             Alaska| United States|
|   4439| 33|     Adults|        Ulm|            Unknown|        Canada|
|   5669| -1|    Unknown|    Chicago|           Illinois| United States|
|   6025| -1|    Unknown|  San Diego|         California| United States|
|   6382| -1|    Unknown| Whitehorse|    Yukon Terr

In [207]:
users_df.groupBy("country").count().orderBy("count").show(147)

+--------------------+-----+
|             country|count|
+--------------------+-----+
|    Marshall Islands|    1|
|         Puerto Rico|    1|
|               Burma|    1|
|          Tajikistan|    1|
|             Algeria|    1|
|               Syria|    1|
|              Uganda|    1|
|              Zambia|    1|
|            Ethiopia|    1|
|               Gabon|    1|
|               Samoa|    1|
|              Guinea|    1|
|                Cuba|    1|
|             Lebanon|    1|
|              Gambia|    1|
|        Turkmenistan|    1|
|               Tonga|    1|
|          Bangladesh|    1|
|          Madagascar|    1|
|     Solomon Islands|    1|
|             Senegal|    1|
|        Burkina Faso|    1|
|         El Salvador|    1|
|               Palau|    1|
|             Bahrain|    1|
|Bosnia And Herzeg...|    1|
|             Holland|    1|
|               Aruba|    2|
|               Niger|    2|
|             Estonia|    2|
|           Sri Lanka|    2|
|             

In [208]:
books_df.show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+---------+--------+
|      isbn|          book_title|         book_author|year_of_publication|           publisher|            category|pagecount|language|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+---------+--------+
|0002234947|Miss Hobbema Pageant|       W.P. Kinsella|               1990|Harpercollins Juv...|       Short stories|      200|      en|
|0007640617|  Energize Your Life|          Nic Rowley|               2002| Thorsons Publishers|    Health & Fitness|      248| unknown|
|0020444400|The voyage of the...|          C. S Lewis|               1986|             Collier|             Fantasy|      210|      en|
|0025483307|Stranger in Two W...|         Jean Harris|               1986|Simon &amp; Schuster|      Social Science|      388|      en|
|0028622480|The First-Book Ma...|       Jason Sh

In [209]:
from pyspark.sql.types import IntegerType , StringType

#changing the data type of the columns
books_df = books_df.withColumn("year_of_publication", F.col("year_of_publication").cast(IntegerType()))\
.withColumn("pagecount", F.col("pagecount").cast(IntegerType()))\
.withColumn("language", F.col("language").cast(StringType()))\
.withColumn("category", F.col("category").cast(StringType()))\
.withColumn("publisher", F.col("publisher").cast(StringType()))\
.withColumn("book_author", F.col("book_author").cast(StringType()))\
.withColumn("book_title", F.col("book_title").cast(StringType()))\
.withColumn("isbn", F.col("isbn").cast(StringType()))

In [210]:
language_list = ['en','de','fr','es','it','nl','ca','pt-BR','sv','pl','fi','ms','gl','el','da','ru','ja','la','iw','ko','ar','cs','cy','no','zh-CN','eu','un','bg','sk','tr','lt','fa']

In [211]:
books_df = books_df.withColumn("language", F.when(F.col("language").isin(language_list),F.col("language")).otherwise("Unknown"))

In [212]:
books_df.groupBy("language").count().orderBy("count",ascending=False).show(147)

+--------+------+
|language| count|
+--------+------+
|      en|224750|
| Unknown| 11585|
|      de|  9592|
|      fr|  5112|
|      es|  5091|
|      it|   598|
|      nl|   257|
|      ca|   253|
|   pt-BR|   182|
|      sv|    34|
|      pl|    30|
|      fi|    30|
|      ms|    24|
|      gl|    23|
|      el|    16|
|      da|    15|
|      ru|    13|
|      ja|    13|
|      la|    11|
|      iw|     9|
|      ar|     9|
|      ko|     9|
|      cs|     7|
|      cy|     7|
|      no|     6|
|   zh-CN|     5|
|      eu|     5|
|      un|     5|
|      bg|     3|
|      sk|     2|
|      lt|     2|
|      tr|     2|
|      fa|     2|
+--------+------+



In [213]:
books_df = books_df.withColumn("year_of_publication", F.when(F.col("year_of_publication")> 2020, value = 0).otherwise(F.col("year_of_publication")))

In [214]:
# if null, replace with 0
books_df = books_df.na.fill(0)

In [215]:
books_df.groupBy("year_of_publication").count().orderBy("year_of_publication",ascending=False).show(147)

+-------------------+-----+
|year_of_publication|count|
+-------------------+-----+
|               2020|    3|
|               2012|    1|
|               2011|    2|
|               2010|    2|
|               2008|    1|
|               2006|    3|
|               2005|   41|
|               2004| 5557|
|               2003|13675|
|               2002|16736|
|               2001|16445|
|               2000|16366|
|               1999|16588|
|               1998|14969|
|               1997|14145|
|               1996|13347|
|               1995|12826|
|               1994|11195|
|               1993|10121|
|               1992| 9420|
|               1991| 8908|
|               1990| 8234|
|               1989| 7564|
|               1988| 7145|
|               1987| 6193|
|               1986| 5525|
|               1985| 5061|
|               1984| 4703|
|               1983| 4268|
|               1982| 3989|
|               1981| 3126|
|               1980| 2548|
|               1979

In [216]:
books_df.show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+---------+--------+
|      isbn|          book_title|         book_author|year_of_publication|           publisher|            category|pagecount|language|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+---------+--------+
|0002234947|Miss Hobbema Pageant|       W.P. Kinsella|               1990|Harpercollins Juv...|       Short stories|      200|      en|
|0007640617|  Energize Your Life|          Nic Rowley|               2002| Thorsons Publishers|    Health & Fitness|      248| Unknown|
|0020444400|The voyage of the...|          C. S Lewis|               1986|             Collier|             Fantasy|      210|      en|
|0025483307|Stranger in Two W...|         Jean Harris|               1986|Simon &amp; Schuster|      Social Science|      388|      en|
|0028622480|The First-Book Ma...|       Jason Sh

In [217]:
books_for_std = books_df.filter(F.col("pagecount")!= 0)

In [218]:
books_for_std.describe().show()

+-------+--------------------+--------------------+--------------+-------------------+--------------------+--------------------+------------------+--------+
|summary|                isbn|          book_title|   book_author|year_of_publication|           publisher|            category|         pagecount|language|
+-------+--------------------+--------------------+--------------+-------------------+--------------------+--------------------+------------------+--------+
|  count|              246640|              246640|        246640|             246640|              246640|              246640|            246640|  246640|
|   mean|1.0245850152059194E9|            Infinity|          null| 1961.4910476808304|             37547.0|       1.823070765E8| 279.2915869283166|    null|
| stddev|1.4653956507443273E9|                 NaN|          null| 251.78603066926115|                 0.0|2.5782114010288936E8|179.42296044023982|    null|
|    min|          0000913154| A Light in the S...|      D

In [219]:
books_for_std.select(F.stddev ("pagecount")).show()

+----------------------+
|stddev_samp(pagecount)|
+----------------------+
|    179.42296044023982|
+----------------------+



In [220]:
books_df = books_df.withColumn("Decade_of_publication",F.lit("Unknown"))

In [221]:
books_df = books_df.withColumn("Decade_of_publication", F.when(F.col("year_of_publication") <1970 , "pre_70's").otherwise(F.col("Decade_of_publication")))\
.withColumn("Decade_of_publication", F.when((F.col("year_of_publication") >=1970) & (F.col("year_of_publication") <1980), "70's").otherwise(F.col("Decade_of_publication")))\
.withColumn("Decade_of_publication", F.when((F.col("year_of_publication") >=1980) & (F.col("year_of_publication") <1990), "80's").otherwise(F.col("Decade_of_publication")))\
.withColumn("Decade_of_publication", F.when((F.col("year_of_publication") >=1990) & (F.col("year_of_publication") <2000), "90's").otherwise(F.col("Decade_of_publication")))\
.withColumn("Decade_of_publication", F.when(F.col("year_of_publication") >=2000, "00's").otherwise(F.col("Decade_of_publication")))

In [222]:
books_df = books_df.withColumn("BookLenghtGroups",F.lit("Unknown"))

In [223]:
books_df = books_df.withColumn("BookLenghtGroups", F.when((F.col("pagecount") <150) & (F.col("pagecount")> 1), 'short').otherwise(F.col("BookLenghtGroups")))\
.withColumn("BookLenghtGroups", F.when((F.col("pagecount") >=150) & (F.col("pagecount") <450), 'Average').otherwise(F.col("BookLenghtGroups")))\
.withColumn("BookLenghtGroups", F.when(F.col("pagecount") >=450, 'Long').otherwise(F.col("BookLenghtGroups")))

In [224]:
books_df.show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+---------+--------+---------------------+----------------+
|      isbn|          book_title|         book_author|year_of_publication|           publisher|            category|pagecount|language|Decade_of_publication|BookLenghtGroups|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+---------+--------+---------------------+----------------+
|0002234947|Miss Hobbema Pageant|       W.P. Kinsella|               1990|Harpercollins Juv...|       Short stories|      200|      en|                 90's|         Average|
|0007640617|  Energize Your Life|          Nic Rowley|               2002| Thorsons Publishers|    Health & Fitness|      248| Unknown|                 00's|         Average|
|0020444400|The voyage of the...|          C. S Lewis|               1986|             Collier|             Fantasy|      210

In [225]:
books_df.describe().show()

+-------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+--------+---------------------+----------------+
|summary|                isbn|          book_title|         book_author|year_of_publication|           publisher|            category|         pagecount|language|Decade_of_publication|BookLenghtGroups|
+-------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+--------+---------------------+----------------+
|  count|              257702|              257702|              257702|             257702|              257702|              257702|            257702|  257702|               257702|          257702|
|   mean|1.0301175105925251E9|            Infinity|                null|  1959.610080635773|  4025.1714285714284| 2.804892753846154E7|267.30284204235903|    null|                 null|        

In [226]:
books_df.groupBy("pagecount").count().orderBy("pagecount",ascending=True).show(800)

+---------+-----+
|pagecount|count|
+---------+-----+
|        0|11062|
|        1|    8|
|        2|    1|
|        3|    4|
|        4|   10|
|        5|   21|
|        6|   32|
|        7|   10|
|        8|   91|
|        9|   16|
|       10|  360|
|       11|   11|
|       12|  428|
|       13|   15|
|       14|  190|
|       15|   36|
|       16|  349|
|       17|   20|
|       18|  132|
|       19|   23|
|       20|  220|
|       21|   58|
|       22|  159|
|       23|  133|
|       24| 1942|
|       25|   85|
|       26|  200|
|       27|   55|
|       28|  288|
|       29|  142|
|       30|  359|
|       31|  333|
|       32| 4252|
|       33|   61|
|       34|   84|
|       35|   48|
|       36|  131|
|       37|   39|
|       38|   58|
|       39|   61|
|       40|  522|
|       41|   86|
|       42|  104|
|       43|   82|
|       44|  221|
|       45|  130|
|       46|  164|
|       47|  233|
|       48| 1316|
|       49|   35|
|       50|   74|
|       51|   84|
|       52

In [227]:
books_df.groupBy("category").count().orderBy("count",ascending=False).show(300)

+--------------------+-----+
|            category|count|
+--------------------+-----+
|             Fiction|83591|
|               other|27229|
|    Juvenile Fiction|14237|
|Biography & Autob...| 7193|
|            Religion| 5038|
|             History| 4748|
| Juvenile Nonfiction| 4273|
|      Social Science| 3255|
|Business & Economics| 2805|
|             Cooking| 2647|
|               Humor| 2351|
| Body, Mind & Spirit| 2285|
|    Health & Fitness| 2281|
|Family & Relation...| 2165|
|           Computers| 2031|
|              Travel| 1811|
|          Psychology| 1658|
|           Self-Help| 1631|
|             Science| 1594|
|              Poetry| 1553|
|  Literary Criticism| 1394|
|                 Art| 1333|
| Sports & Recreation| 1234|
|          Philosophy| 1142|
|Detective and mys...| 1138|
|           Reference| 1082|
|   Political Science| 1075|
|              Nature| 1074|
|Language Arts & D...| 1018|
|               Drama|  990|
|     Performing Arts|  977|
|           Ed

In [242]:
books_df = books_df.withColumn("book_author" , F.when(F.col("isbn")== '0425054594' , "E.E. Smith").otherwise(F.col("book_author")))\
.withColumn("book_author" , F.when(F.col("isbn")== '0425054586' , "E.E. Smith").otherwise(F.col("book_author")))\
.withColumn("book_author" , F.when(F.col("isbn")== '0425054608' , "E.E. Smith").otherwise(F.col("book_author")))\
.withColumn("book_author" , F.when(F.col("isbn")== '0425062449' , "E.E. Smith").otherwise(F.col("book_author")))

In [246]:
books_df.filter(F.col("isbn") == "0425062449").show(truncate = False)

+----------+------------------+-----------+-------------------+-----------------+--------+---------+--------+---------------------+----------------+
|isbn      |book_title        |book_author|year_of_publication|publisher        |category|pagecount|language|Decade_of_publication|BookLenghtGroups|
+----------+------------------+-----------+-------------------+-----------------+--------+---------+--------+---------------------+----------------+
|0425062449|Subspace Encounter|E.E. Smith |1983               |Berkley Pub Group|Fiction |198      |en      |80's                 |Average         |
+----------+------------------+-----------+-------------------+-----------------+--------+---------+--------+---------------------+----------------+



In [247]:
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window

ratings_df = ratings_df.withColumn(
    "indexSK",
    row_number().over(Window.orderBy(monotonically_increasing_id()))+99
)

users_df= users_df.withColumn(
    "user_id_SK",
    row_number().over(Window.orderBy(monotonically_increasing_id()))+99
)

books_df= books_df.withColumn(
    "isbn_SK",
    row_number().over(Window.orderBy(monotonically_increasing_id()))+99
)

ratings_df = ratings_df.select("indexSK","isbn","user_id","book_ratings")
users_df = users_df.select("user_id_SK","user_id","Age","AgeGroups","city","state","country")
books_df= books_df.select("isbn_SK","isbn","book_title","book_author","year_of_publication","publisher","category","pagecount","language","BookLenghtGroups","Decade_of_publication")

In [248]:
ratings_df.show()

+-------+----------+-------+------------+
|indexSK|      isbn|user_id|book_ratings|
+-------+----------+-------+------------+
|    100|0843920262| 276762|           0|
|    101|3596122279| 276936|           5|
|    102|0713911131| 277378|           0|
|    103|067091021X| 277427|           0|
|    104|0836221311| 277427|           8|
|    105|0553279556| 277466|           0|
|    106|0449907481| 277554|           0|
|    107|0879052473| 277562|           8|
|    108|0330484338| 277901|           8|
|    109|0451206460| 277919|           9|
|    110|1557850003| 277929|           0|
|    111|0515135968| 278026|           8|
|    112|0375727345| 278122|          10|
|    113|0385319959| 278314|           7|
|    114|0070145121| 278418|           0|
|    115|0152006044| 278418|           0|
|    116|0307030849| 278418|           0|
|    117|0307101452| 278418|           0|
|    118|0373117868| 278418|           5|
|    119|0373122462| 278418|           0|
+-------+----------+-------+------

In [249]:
books_df.show()

+-------+----------+--------------------+--------------------+-------------------+--------------------+--------------------+---------+--------+----------------+---------------------+
|isbn_SK|      isbn|          book_title|         book_author|year_of_publication|           publisher|            category|pagecount|language|BookLenghtGroups|Decade_of_publication|
+-------+----------+--------------------+--------------------+-------------------+--------------------+--------------------+---------+--------+----------------+---------------------+
|    100|0002234947|Miss Hobbema Pageant|       W.P. Kinsella|               1990|Harpercollins Juv...|       Short stories|      200|      en|         Average|                 90's|
|    101|0007640617|  Energize Your Life|          Nic Rowley|               2002| Thorsons Publishers|    Health & Fitness|      248| Unknown|         Average|                 00's|
|    102|0020444400|The voyage of the...|          C. S Lewis|               1986|   

In [250]:
users_df.show()

+----------+-------+---+-----------+-----------+-------------------+--------------+
|user_id_SK|user_id|Age|  AgeGroups|       city|              state|       country|
+----------+-------+---+-----------+-----------+-------------------+--------------+
|       100|   2355| -1|    Unknown|   Gatineau|             Quebec|        Canada|
|       101|   2936| 30|     Adults|     London|             London|United Kingdom|
|       102|   2964| -1|    Unknown|   Tualatin|             Oregon| United States|
|       103|   3282| 52|     Adults|   Didsbury|            Alberta|        Canada|
|       104|   3641| 34|     Adults|King George|           Virginia| United States|
|       105|   3645| -1|    Unknown|   Siegburg|Nordrhein-westfalen|       Germany|
|       106|   4193| -1|    Unknown|    Douglas|             Alaska| United States|
|       107|   4439| 33|     Adults|        Ulm|            Unknown|        Canada|
|       108|   5669| -1|    Unknown|    Chicago|           Illinois| United 

In [251]:
ratings_df.coalesce(1).write.option("header" ,"True").csv("/content/drive/MyDrive/spark/ratings_df.csv","overwrite",header =True)

In [252]:
users_df.coalesce(1).write.option("header" ,"True").csv("/content/drive/MyDrive/spark/users_df.csv","overwrite",header =True)

In [253]:
books_df.coalesce(1).write.option("header" ,"True").csv("/content/drive/MyDrive/spark/books_df.csv","overwrite",header =True)

In [171]:
# Stop the session --> similar to saving our doings in spark layer
spark.stop()

In [172]:
# checking if the isbn is real or not (with isbnlib) and droping the row if it is not real
#from isbnlib import notisbn
#list_not_isbn =[]
#for i in range (len(df.select("isbn").collect())):
    #if notisbn(df.select("isbn").collect()[i][0]):
        #list_not_isbn.append(df.select("isbn").collect()[i][0])
        # df = df.where("isbn!=i")

In [173]:
#list_not_isbn